NOTE: I'm providing a 3 implementations below with increasing complexity because if you only use the standard circuit that gives you p(0) = cos²(π·α), there’s an inherent ambiguity. This function is symmetric, meaning it can’t distinguish between α and 1 – α. The circuit “sees” only the effective phase modulo 1, and for any measurement you get two possible α values (one in [0, 0.5] and its complement in [0.5, 1]).

To resolve this ambiguity and find α uniquely, one would need to add an additional measurement that can capture the sign (or full phase) information. One approach is to perform an extra measurement—such as a tomography-like measurement that obtains the expectation values of both Pauli‑X and Pauli‑Y on the control qubit—to reconstruct the full phase unambiguously.

In [17]:
import pennylane as qml
import numpy as np

# Generate a random alpha in [0, pi/2]
alpha = np.random.uniform(0, np.pi/2)
print("Actual alpha:", alpha)

# Create a quantum device with 2 qubits and a sufficient number of shots for statistics
dev = qml.device("default.qubit", wires=2, shots=10000)

@qml.qnode(dev)
def phase_estimation_circuit(alpha):
    # Initialize the target qubit in |1⟩
    qml.PauliX(wires=1)
    
    # Put the control qubit in a superposition
    qml.Hadamard(wires=0)
    
    # Apply the controlled phase gate with phase = 2*pi*alpha
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    
    # Apply another Hadamard on the control qubit to complete the interference
    qml.Hadamard(wires=0)
    
    # Return a sample measurement on the control qubit
    return qml.sample(wires=0)

# Run the circuit to obtain measurement outcomes
samples = phase_estimation_circuit(alpha)

# Estimate the probability of measuring 0 (note: samples is an array of 0's and 1's)
p0 = np.mean(samples == 0)
print("Estimated probability of measuring 0:", p0)

# Invert the relation: p0 = cos²(pi*alpha)
# Since alpha*pi is in [0, pi/2] (cosine is nonnegative), we have:
#   cos(pi*alpha) = sqrt(p0)
#   => alpha = arccos(sqrt(p0)) / pi
alpha_est = np.arccos(np.sqrt(p0)) / np.pi
print("Estimated alpha:", alpha_est)


Actual alpha: 1.2317623471715446
Estimated probability of measuring 0: 0.5569
Estimated alpha: 0.23184884518382454


In [22]:
import pennylane as qml
import numpy as np

# Generate a random alpha in [0, π/2]
alpha = np.random.uniform(0, np.pi/2)
print("True alpha:", alpha)

# Compute the effective phase (alpha modulo 1)
alpha_effective = alpha % 1
print("Effective alpha (alpha mod 1):", alpha_effective)

# Create a quantum device with 2 qubits and many shots
dev = qml.device("default.qubit", wires=2, shots=1000)

@qml.qnode(dev)
def phase_estimation_circuit(alpha):
    # Prepare target qubit in |1⟩
    qml.PauliX(wires=1)
    # Create superposition on control qubit
    qml.Hadamard(wires=0)
    # Apply the controlled phase gate with phase = 2π*alpha
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    # Final Hadamard on control qubit
    qml.Hadamard(wires=0)
    # Return sample measurement on the control qubit
    return qml.sample(wires=0)

# Run the circuit and calculate the probability of measuring 0
samples = phase_estimation_circuit(alpha)
p0 = np.mean(samples == 0)
print("Estimated probability of measuring 0:", p0)

# Invert the relation p0 = cos²(π·alpha_effective)
alpha_est = np.arccos(np.sqrt(p0)) / np.pi
# The inversion gives a value in [0, 0.5]. The complementary possibility is:
alpha_alt = 1 - alpha_est

print("Estimated effective alpha from the circuit could be either:")
print("Option 1 (direct inversion):", alpha_est)
print("Option 2 (complement):", alpha_alt)


True alpha: 0.05232675787901816
Effective alpha (alpha mod 1): 0.05232675787901816
Estimated probability of measuring 0: 0.976
Estimated effective alpha from the circuit could be either:
Option 1 (direct inversion): 0.049511766179566125
Option 2 (complement): 0.9504882338204339


In [25]:
import pennylane as qml
import numpy as np

# Generate a random alpha in [0, π/2]
alpha = np.random.uniform(0, np.pi/2)
print("True alpha:", alpha)

# Compute the effective alpha modulo 1 (the circuit is sensitive only to the fractional part)
alpha_effective = alpha % 1
print("Effective alpha (alpha mod 1):", alpha_effective)

# Create devices for both circuits
# Standard circuit device for comparison
dev_standard = qml.device("default.qubit", wires=2, shots=1000)
# Tomography device to extract phase information
dev_tomo = qml.device("default.qubit", wires=2, shots=1000)

@qml.qnode(dev_standard)
def standard_circuit(alpha):
    # Initialize the target qubit in |1⟩
    qml.PauliX(wires=1)
    # Create a superposition on the control qubit
    qml.Hadamard(wires=0)
    # Apply the controlled phase gate with phase = 2π*alpha
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    # Final Hadamard on control qubit
    qml.Hadamard(wires=0)
    # Return a sample measurement on the control qubit
    return qml.sample(wires=0)

@qml.qnode(dev_tomo)
def tomography_circuit(alpha):
    # Initialize the target qubit in |1⟩
    qml.PauliX(wires=1)
    # Create a superposition on the control qubit
    qml.Hadamard(wires=0)
    # Apply the controlled phase gate with phase = 2π*alpha
    qml.CPhase(2 * np.pi * alpha, wires=[0, 1])
    # Do not apply the final Hadamard so that the state remains:
    # (|0⟩ + e^(i2π*alpha)|1⟩)/√2.
    # Measure the expectation values in the X and Y bases
    exp_x = qml.expval(qml.PauliX(0))
    exp_y = qml.expval(qml.PauliY(0))
    return exp_x, exp_y

# Run the standard circuit to obtain p(0)
samples = standard_circuit(alpha)
p0 = np.mean(samples == 0)
alpha_est_standard = np.arccos(np.sqrt(p0)) / np.pi
print("\nStandard circuit:")
print("Estimated probability of measuring 0:", p0)
print("Estimated effective alpha from p(0):", alpha_est_standard)
print("Complementary possibility:", 1 - alpha_est_standard)

# Run the tomography circuit to extract the full phase
expX, expY = tomography_circuit(alpha)
phi = np.arctan2(expY, expX)  # This gives 2π*alpha, with the correct sign.
# Ensure phi is nonnegative (since our effective phase is in [0, 1])
phi = phi if phi >= 0 else (phi + 2 * np.pi)
alpha_est_tomo = phi / (2 * np.pi)
print("\nTomography circuit:")
print("⟨X⟩ =", expX, " ⟨Y⟩ =", expY)
print("Extracted phase (phi):", phi)
print("Estimated effective alpha from tomography:", alpha_est_tomo)


True alpha: 0.40574895179767556
Effective alpha (alpha mod 1): 0.40574895179767556

Standard circuit:
Estimated probability of measuring 0: 0.082
Estimated effective alpha from p(0): 0.40755585848699366
Complementary possibility: 0.5924441415130064

Tomography circuit:
⟨X⟩ = -0.784  ⟨Y⟩ = 0.544
Extracted phase (phi): 2.5349875449952664
Estimated effective alpha from tomography: 0.4034557984623851
